Ridge regression
---

Exercise - Basic feature engineering for the house prices data
---

> **Exercise**: (A) load the house prices data from `house-prices.csv` (B) plot the distribution of the continuous variables using histograms: you should see that many have a skewed one (C) create a `preprocess(df)` function which performs (C.1) one-hot encoding (C.2) fill missing values (C.3) apply a **log-transform** to every continuous feature and (C.4) add their **polynomial features** of degree 2, 3 and 0.5 (square root). Finally (D) create the X/y numpy arrays - use the `np.log10()` of the sale price as the target variable.

**Hint**: The logarithm of zero doesn't exist, so we have to make sure that there are no zero values in the continuous columns when applying the log-transform. To achieve this, we can use `np.log(x+1)` (or simply `np.log1p(x)` which is equivalent) which will leave zero values untransformed i.e. `log(1)=0`

In [ ]:
# (A) Load the data
data_df = ???

In [ ]:
# Continuous features from the documentation
continuous = [
    'Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
    'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
    'Gr Liv Area', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
    '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val']

In [ ]:
# (B) Plot the distribution of the continuous variables using histograms
???

In [ ]:
# (C) Preprocessing function
def preprocess(df):
    # Work on a copy
    df = df.copy()
    
    # (C.1) One-hot encoding
    ???
    
    # (C.2) Fill missing values
    ???
    
    # (C.3) Apply log-transform
    ???
    
    # (C.4) Add polynomial features
    ???
        
    return df

preprocessed_df = preprocess(data_df)
preprocessed_df.head()

In [ ]:
# (D) Create X, y
X = ???
y = ???

Exercise - Tune Ridge regression
---

> **Exercise**: Fit a ridge regression model and tune its alpha value using grid search. Use the train/test set methodology with a 50/50 split. Print the optimal alpha value and the test MSE/MAE scores.

**Hint**: Don't forget to standardize the input data.

In [ ]:
???